<a href="https://colab.research.google.com/github/HKang42/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/Harrison_Kang_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

**Load and clean the data**

In [0]:
### YOUR CODE STARTS HERE
import pandas as pd
import numpy as np
import scipy.stats as stats

In [0]:
# Create column headers

# Grab column names from the "house-votes-84.names" file hosted in the same directory. Then split based on colons
cols = "Class: handicapped-infants: water-project-cost-sharing: adoption-of-the-budget-resolution: physician-fee-freeze: el-salvador-aid: religious-groups-in-schools: anti-satellite-test-ban: aid-to-nicaraguan-contras: mx-missile: immigration: synfuels-corporation-cutback: education-spending: superfund-right-to-sue: crime: duty-free-exports: export-administration-act-south-africa:"

cols = cols.split(': ')

In [0]:
# Get data

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

df = pd.read_csv(url, names = cols)


In [4]:
df.head()

,Class,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa:
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [5]:
# replace string values with numeric ones

df = df.replace({"y" : 1, "n" : 0, "?" : np.NaN})

df.head()

,Class,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa:
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
# split the data into democrat and republican

dem = df[ df['Class'] == 'democrat']
rep = df[ df['Class'] == 'republican']

**Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01**

Null Hypo: Both democrat and republican support for a given issue is the same

Alternative Hypo: Their support issues are not the same

Confidence Level = 99%   -->   Critical Value = 1%

If p-value < 0.01 , reject the null hypothesis (support is the same)

If p-value > 0.01 , fail to reject the null hypothesis (support might not be the same)

In [7]:
issue = 'anti-satellite-test-ban'

results = stats.ttest_ind(dem[issue], rep[issue], nan_policy='omit')

print ('t-value =\t', results[0])
print ('p-value =\t', results[1])

t-value =	 12.526187929077842
p-value =	 8.521033017443867e-31


Given a t-value of 12.526 and a p-value of 8.52 e-31 (approximately 0), I reject the null hypothesis that democratic and republican support for the "anti-satellite-test-ban" were equal. 

The large and positive t-value suggest that democrats supported the bill significantly more than republicans.

**Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01**

In [8]:
issue = 'superfund-right-to-sue'

results = stats.ttest_ind(dem[issue], rep[issue], nan_policy='omit')

print ('t-value =\t', results[0])
print ('p-value =\t', results[1])

t-value =	 -13.51064251060933
p-value =	 1.2278581709672758e-34


Given a t-value of -13.51 and a p-value of 1.22 e-34 (approximately 0), I reject the null hypothesis that democratic and republican support for the "superfund-right-to-sue" were equal. 

The large and negative t-value suggest that republicans supported the bill significantly more than democrats.

**Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1**

In [9]:
issue = 'water-project-cost-sharing'

results = stats.ttest_ind(dem[issue], rep[issue], nan_policy='omit')

print ('t-value =\t', results[0])
print ('p-value =\t', results[1])

t-value =	 -0.08896538137868286
p-value =	 0.9291556823993485


Given a t-value of -0.0890 and a p-value of 0.929, I fail to reject the null hypothesis that democratic and republican support for the "water-project-cost-sharing" were equal. 

The small t-value and large p-value suggest that there was little difference in how much republicans and democrats supported the issue.

**Stretch goals:**

**Refactor your code into functions so it's easy to rerun with arbitrary variables**

and

**Try and write a function that will help you do all of the tests automatically (republican vs. democratic for all issues)** (suggested in class)

In [0]:
# FUNCTION IS HERE

"""
Function arguments are: 

  - Dataframe imported above 
      - Dataframe with one column for 2 groups and other column(s) for their votes on topics. Votes must be numerical (0 or 1)

  - Name of the party column (which group does each observation/person belong to)

  - Name of the issue being voted on

  - Critical Value
      - Critical Value = 1 - Desired Certainty

  - Truncate = True/False
      - Truncate t-values and p-values if desired.
      - default setting is True


Function returns a dictionary containing:
  - The name of the issue being voted on

  - The t-value for that issue
      - will be truncated to 3 decimal points by default

  - The p-value for that issue
      - will be truncated to 3 decimal points by default

  - Which group supported the issue more
    - if there was no significant difference between votes, it returns "Same"

"""
import decimal 
import scipy.stats as stats
import pandas as pd

def t_tester (df , party_col, issue, certainty, truncate=True):

  # get the name of the different groups
  party1 = df[party_col].value_counts().index[0]
  party2 = df[party_col].value_counts().index[1]

  # create separate dataframes for each group
  df_party1 = df[ df[party_col] == party1]
  df_party2 = df[ df[party_col] == party2]

  # Do the T-Test
  results = stats.ttest_ind(df_party1[issue], df_party2[issue], nan_policy='omit')
  
  # Record the results of the T-Test
  t_val = results[0]
  p_val = results[1]
  

  # Truncate numbers if desired
  if truncate == True:
  
    #round the t_val
    t_val = round(t_val,3)

    #round the p_value
    p_val = round(p_val, 3)

    # cut off zeroes if rounded p_value is 0
    if p_val == 0:
        p_val = 0

  # Check the p-value and t-value to determine if support differed between parties. 
  # If so, determine which party supported the issue more.
  favor = ''

  if p_val >= certainty:
    favor = 'Same'
  
  if p_val < certainty:
    if t_val > 0:
      favor = party1

    elif t_val < 0:
      favor = party2


  results_dict = {'issue' : issue, 't-value' : t_val, "p-value" : p_val, "Support" : favor}

  return results_dict

In [0]:
# Create dataframe to store our results
t_test = pd.DataFrame(columns= ['issue', 't-value', 'p-value', 'Support'])

# set desired Critical Value
certainty = 0.05

#Loop through each issue in our dataframe to get results for each one.
for issue in df.columns[1:]:
  
  row = t_tester(df, 'Class', issue , certainty)

  t_test = t_test.append(row, ignore_index=True)

In [12]:
# display results
t_test

,issue,t-value,p-value,Support
0,handicapped-infants,9.205,0,democrat
1,water-project-cost-sharing,-0.089,0.929,Same
2,adoption-of-the-budget-resolution,23.213,0,democrat
3,physician-fee-freeze,-49.367,0,republican
4,el-salvador-aid,-21.137,0,republican
5,religious-groups-in-schools,-9.738,0,republican
6,anti-satellite-test-ban,12.526,0,democrat
7,aid-to-nicaraguan-contras,18.052,0,democrat
8,mx-missile,16.438,0,democrat
9,immigration,-1.736,0.083,Same
